In [2]:
%cd ../..

c:\Users\HuyLQ15_CTV\Desktop\model_monitoring


In [3]:
from datetime import time, datetime
import pandas as pd
import numpy as np
import requests
import zipfile
import io
from datetime import datetime, time
from evidently import ColumnMapping

content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip", verify=False).content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday'], index_col='dteday')
raw_data.index = raw_data.apply(
    lambda row: datetime.combine(row.name, time(hour=int(row['hr']))), axis = 1)

c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ho-proxy02.tpb.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [4]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']
column_mapping = ColumnMapping()
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-31 00:00:00']
current = raw_data.loc['2011-02-01 00:00:00':'2011-07-28 23:00:00']
current['time_stamp'] = current.index
reference['time_stamp'] = reference.index

C:\Users\HuyLQ15_CTV\AppData\Local\Temp\ipykernel_18556\4123119252.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current['time_stamp'] = current.index
C:\Users\HuyLQ15_CTV\AppData\Local\Temp\ipykernel_18556\4123119252.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reference['time_stamp'] = reference.index


# Utils Function

In [5]:
from nannyml.distribution import CategoricalDistributionCalculator

categorical_dist = CategoricalDistributionCalculator(
    column_names=['workingday'],
    timestamp_column_name = 'time_stamp',
    chunk_period='M',
)
categorical_dist.fit(reference).calculate(current).filter(period='all', column_names=['workingday']).plot(kind='distribution')

c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\nannyml\chunk.py:181: UserWarning: The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.
  warnings.warn(
c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\nannyml\chunk.py:181: UserWarning: The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.
  warnings.warn(


In [6]:
from nannyml.distribution import ContinuousDistributionCalculator

continuous_dist = ContinuousDistributionCalculator(
    column_names=['temp'],
    timestamp_column_name = 'time_stamp',
    chunk_period='M',
)
fig = continuous_dist.fit(reference).calculate(current).filter(period='all', column_names='temp').plot(kind='distribution')
fig.data[0]

c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\nannyml\chunk.py:181: UserWarning:

The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.

c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\nannyml\chunk.py:181: UserWarning:

The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.



Scatter({
    'hoverinfo': 'skip',
    'line': {'color': <Colors.BLUE_SKY_CRAYOLA: '#00c8e5'>, 'width': 1},
    'mode': 'lines',
    'name': 'Reference',
    'showlegend': False,
    'x': array(['2010-12-31T23:15:52.389722905', '2010-12-31T21:23:15.086181636',
                '2010-12-31T14:17:15.621890120', '2010-12-30T21:59:44.866914464',
                '2010-12-29T20:40:44.302621879', '2010-12-28T17:32:28.171963638',
                '2010-12-27T21:39:15.584714684', '2010-12-27T13:25:13.211330664',
                '2010-12-27T14:30:32.059992198', '2010-12-27T16:50:45.192592149',
                '2010-12-27T09:49:04.204824501', '2010-12-26T10:00:37.366545912',
                '2010-12-24T17:49:19.071052544', '2010-12-22T11:32:27.601455514',
                '2010-12-19T09:21:26.149682595', '2010-12-15T04:22:17.280489031',
                '2010-12-10T07:08:06.182469552', '2010-12-06T01:12:49.542035495',
                '2010-12-03T19:08:31.624843128', '2010-12-03T12:22:02.198860563',
 

In [7]:
import evidently.options as evidently_options
from typing import List, Optional
class ColorOptions(evidently_options.ColorOptions):
    primary_gradient_color: List[str] = ['#ed0400','rgba(255, 0, 0, 0.3)']
    secondary_gradient_color: List[str] = ['#00c8e5', 'rgba(0.0, 0.7843137254901961, 0.8980392156862745, 0.2)']

In [8]:
from nannyml.drift.univariate import UnivariateDriftCalculator
import plotly.graph_objects as go
from typing import Union
def drift_distribution_num_plot(
    reference: pd.DataFrame, 
    current: pd.DataFrame, 
    col: str, 
    timestamp_col: str,
    num_drift_methods: List[str]=['kolmogorov_smirnov'],
    color_option: ColorOptions = ColorOptions(),
    chunk_period: str = "M",
) -> go.Figure:
    drift = UnivariateDriftCalculator(
        column_names=col,
        treat_as_numerical=col,
        timestamp_column_name=timestamp_col,
        continuous_methods=num_drift_methods,
        chunk_period=chunk_period, 
    ).fit(reference_data=reference)
    fig = drift.calculate(data=current).plot(kind='distribution')

    bg_color = fig.layout.plot_bgcolor or fig.layout.paper_bgcolor
    axis_line_color = 'white' if bg_color in ['black', '#000000', '#111111'] else 'black'
    fig.update_layout(
        xaxis=dict(
            type="date",
            title="Time",
            showline=True,
            linecolor=axis_line_color,
        ),
        yaxis=dict(
            title="Value",
            showline=True,
            linecolor=axis_line_color,
        ),
        showlegend=True,
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
    )
    return fig

def drift_distribution_cat_plot(
    reference: pd.DataFrame, 
    current: pd.DataFrame, 
    col: str, 
    timestamp_col: str,
    cat_drift_method: Union[List[str], str] = ['jensen_shannon'],
    color_option: ColorOptions = ColorOptions(),
    chunk_period: str = "M",
) -> go.Figure:
    drift = UnivariateDriftCalculator(
        column_names=col,
        treat_as_categorical=col,
        timestamp_column_name=timestamp_col,
        categorical_methods=cat_drift_method,
        chunk_period=chunk_period, 
    ).fit(reference_data=reference)
    fig = drift.calculate(data=current).plot(kind='distribution')
    bg_color = fig.layout.plot_bgcolor or fig.layout.paper_bgcolor
    axis_line_color = 'white' if bg_color in ['black', '#000000', '#111111'] else 'black'
    fig.update_layout(
    xaxis=dict(
        type="date",
        title="Time",
        showline=True,
        linecolor=axis_line_color,
    ),
    yaxis=dict(
        title="Value",
        showline=True,
        linecolor=axis_line_color,
    ),
    showlegend=True,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    )
    for annotation in fig.layout.annotations:
        annotation.font.color = None
    return fig


In [10]:
from source.mlops_observation.utils.visualization import drift_distribution_cat_plot
# t = drift_distribution_num_plot(
#     reference=reference,
#     current=current,
#     col='temp',
#     timestamp_col='time_stamp'
# ).show()
t = drift_distribution_cat_plot(
    reference=reference,
    current=current,
    col='season',
    timestamp_col='time_stamp',
    kind='drift'
)
t.show()

c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\nannyml\chunk.py:181: UserWarning:

The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.

c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\nannyml\chunk.py:181: UserWarning:

The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.



In [20]:
t.data[5]

Scatter({
    'hoverinfo': 'skip',
    'legendgroup': 'thresh',
    'line': {'color': <Colors.RED_IMPERIAL: '#DD4040'>, 'dash': 'dash', 'width': 2},
    'mode': 'lines',
    'name': 'Threshold',
    'showlegend': True,
    'x': array([datetime.datetime(2011, 1, 1, 0, 0),
                datetime.datetime(2011, 1, 31, 23, 59, 59, 999999)], dtype=object),
    'xaxis': 'x',
    'y': array([0., 0.]),
    'yaxis': 'y'
})

In [ ]:
from nannyml.distribution import ContinuousDistributionCalculator
from nannyml.distribution import CategoricalDistributionCalculator
import plotly.graph_objects as go
from typing import List, Union, Literal

def period_distribution_num_plot(
    reference: pd.DataFrame, 
    current: pd.DataFrame, 
    col: str, 
    timestamp_col: str,
    color_option: ColorOptions = ColorOptions(),
    chunk_period: str = "M",
) -> go.Figure:
    # if col_type == 'num':
    continuous_dist = ContinuousDistributionCalculator(
    column_names=col,
    timestamp_column_name = timestamp_col,
    chunk_period=chunk_period,
    )
    fig = continuous_dist.fit(reference) \
        .calculate(current) \
        .filter(period='all', column_names=col) \
        .plot(kind='distribution')

    found_analysis = False
    for trace in fig.data:  
        if 'x' in trace:
            trace['x'] = pd.to_datetime(trace['x'], errors='coerce')  # Convert x values to datetime

        if trace.name == "Reference":
            trace.line.color = color_option.secondary_gradient_color[0]

        if trace.name == "Analysis":  # Check if the trace name is "Analysis"
            trace.line.color = color_option.primary_gradient_color[0]  # Update the color to red
            found_analysis = True
        
        if found_analysis == True and isinstance(trace, go.Scatter) and trace.fill =='tonexty':
            trace.fillcolor = color_option.primary_gradient_color[1]
        else:
            trace.fillcolor = color_option.secondary_gradient_color[1]


    bg_color = fig.layout.plot_bgcolor or fig.layout.paper_bgcolor
    axis_line_color = 'white' if bg_color in ['black', '#000000', '#111111'] else 'black'
    fig.update_layout(
        xaxis=dict(
            type="date",
            title="Time",
            showline=True,
            linecolor=axis_line_color,
        ),
        yaxis=dict(
            title="Value",
            showline=True,
            linecolor=axis_line_color,
        ),
        showlegend=True,
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
    )
    return fig

In [ ]:
def period_distribution_cat_plot(
    reference: pd.DataFrame, 
    current: pd.DataFrame, 
    col: str, 
    timestamp_col: str,
    color_option: ColorOptions = ColorOptions(),
    chunk_period: str = "M",
) -> go.Figure:
    categorical_dist = CategoricalDistributionCalculator(
        column_names=col,
        timestamp_column_name = timestamp_col,
        chunk_period=chunk_period,
    )
    fig = categorical_dist.fit(reference) \
        .calculate(current) \
        .filter(period='all', column_names=col) \
        .plot(kind='distribution')
    bg_color = fig.layout.plot_bgcolor or fig.layout.paper_bgcolor
    axis_line_color = 'white' if bg_color in ['black', '#000000', '#111111'] else 'black'
    fig.update_layout(
    xaxis=dict(
        type="date",
        title="Time",
        showline=True,
        linecolor=axis_line_color,
    ),
    yaxis=dict(
        title="Value",
        showline=True,
        linecolor=axis_line_color,
    ),
    showlegend=True,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    )
    for annotation in fig.layout.annotations:
        annotation.font.color = None
    return fig

In [ ]:
categorical_features

In [ ]:
t = period_distribution_cat_plot(
    reference,
    current,
    'workingday',
    'time_stamp'
)
t.show()

In [ ]:
import plotly.graph_objects as go
import pandas as pd


# Assuming `t` contains the traces from period_distribution_plot
t = period_distribution_cat_plot(
    reference, 
    current, 
    col=['season'], 
    timestamp_col='time_stamp'
)
t.show()

# Metrics

In [ ]:
from evidently.renderers.base_renderer import MetricRenderer, default_renderer
from evidently.model.widget import BaseWidgetInfo
from evidently.base_metric import MetricResult, Metric, InputData
from mlops_observation.metric_results.data_drift import CategoricalFeatureDrift, NumericFeatureDrift

from typing import Optional, Dict, List
from typing import Union
from evidently.renderers.base_renderer import MetricRenderer, default_renderer
from evidently.model.widget import BaseWidgetInfo
from evidently.base_metric import InputData, Metric, MetricResult
from evidently.utils.data_operations import process_columns
from evidently import ColumnMapping

from evidently.renderers.html_widgets import ColumnDefinition
from evidently.renderers.html_widgets import RichTableDataRow
from evidently.renderers.html_widgets import RowDetails
from evidently.renderers.html_widgets import CounterData
from evidently.renderers.html_widgets import rich_table_data
from evidently.renderers.html_widgets import header_text
from evidently.renderers.html_widgets import plotly_figure
from evidently.renderers.html_widgets import rich_table_data
from evidently.renderers.html_widgets import table_data
from evidently.renderers.html_widgets import counter
from evidently.utils.visualizations import plot_distr
from evidently.renderers import html_widgets
from evidently.renderers.html_widgets import plotly_data


In [ ]:
class PeriodDataDrifts(MetricResult):
    class Config:
        type_alias = "evidently:metric_result:DataDriftResult"
    numerical_feature_period_results: Optional[Dict[str, NumericFeatureDrift]]
    categorical_feature_period_results: Optional[Dict[str, CategoricalFeatureDrift]]
    fig: Optional[go.Figure]

class DataDriftMetric(Metric[PeriodDataDrifts]):
    """
    time_period (str): Offset aliases https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects
    """
    class Config:
        type_alias = "evidently:metric:DataDriftMetric"
    _time_period: str
    def __init__(self, time_period: str = 'M'):
        self._time_period = time_period
        super().__init__()

    def calculate(self, data: InputData):
        if data.reference_data is None:
            raise ValueError("Reference dataset should be present")
        if data.current_data is None:
            raise ValueError("Current dataset should be present")
        column_mapping = data.column_mapping
        # fig = period_distribution_num_plot(
        #     reference=data.reference_data,
        #     current=data.current_data,
        #     col=column_mapping.numerical_features,
        #     timestamp_col=column_mapping.datetime_features,
        # )
        fig = period_distribution_cat_plot(
            reference=data.reference_data,
            current=data.current_data,
            col=column_mapping.categorical_features,
            timestamp_col=column_mapping.datetime_features,
            chunk_period=self._time_period
        )
        
        return PeriodDataDrifts(fig=fig)
@default_renderer(wrap_type=DataDriftMetric)
class DataDriftRender(MetricRenderer):
    def render_json(self, obj: DataDriftMetric, include_render: bool = False,
        include: "IncludeOptions" = None, exclude: "IncludeOptions" = None,) -> dict:
        result = obj.get_result().get_dict(include_render, include, exclude)
        return result
    def render_html(self, obj: DataDriftMetric) -> List[BaseWidgetInfo]:
        results = obj.get_result()
        color_options = self.color_options
        fig = results.fig
        hist = plotly_figure(title='...',figure=fig)
        # hist = plotly_figure(title='', figure=results.fig)
        return [
            hist
        ]

In [ ]:
from evidently import ColumnMapping

target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']
column_mapping = ColumnMapping()
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features
column_mapping.datetime_features = 'time_stamp'

In [ ]:
from evidently.report import Report
report = Report([DataDriftMetric()])
report.run(reference_data=reference, current_data=current, column_mapping=column_mapping)
report.save_html('1.html')

In [ ]:
from source.mlops_observation.utils.visualization import drift_distribution_cat_plot, drift_distribution_num_plot

t = drift_distribution_num_plot(
    reference=reference,
    current=current,
    col='temp',
    timestamp_col='time_stamp'
).show()
t = drift_distribution_cat_plot(
    reference=reference,
    current=current,
    col='season',
    timestamp_col='time_stamp'
)
t.show()

In [ ]:
from source.mlops_observation.utils.visualization import combine_fig

def combine_drift_each_col_plot(
    reference: pd.DataFrame, 
    current: pd.DataFrame, 
    col_name:str, 
    type_col: Literal['num', 'cat'],
    time_stamp_col : str,
    kind: Literal['drift', 'distribution'],
    num_drift_methods = ['kolmogorov_smirnov', 'jensen_shannon', 'wasserstein'],
    cat_drift_methods = ['chi2', 'jensen_shannon'],
    chunk_period: str = "M"
    ) -> go.Figure:

    drift_figures = []
    
    if kind not in ['drift', 'distribution']:
        raise ValueError("kind must be 'drift' or 'distribution'")

    if type_col == 'num':
        for method in num_drift_methods:
            drift_figures.append(
                drift_distribution_num_plot(
                    reference=reference,
                    current=current,
                    col=col_name,
                    num_drift_method=method,
                    kind=kind,
                    timestamp_col=time_stamp_col,
                    chunk_period=chunk_period)
            )
        drift_combined_fig = combine_fig(drift_figures, num_drift_methods)

    elif type_col == 'cat':
        for method in cat_drift_methods:
            drift_figures.append(
                drift_distribution_cat_plot(
                    reference=reference,
                    current=current,
                    col=col_name,
                    cat_drift_method=method,
                    kind=kind,
                    timestamp_col=time_stamp_col,
                    chunk_period=chunk_period)
            )
        drift_combined_fig = combine_fig(drift_figures, cat_drift_methods)
    else:
        raise ValueError("type_col must be 'num' or 'cat'")
        
    return drift_combined_fig


NameError: name 'Literal' is not defined

In [ ]:
combine_drift_each_col_plot(
    reference, 
    current, 
    col_name='holiday', 
    type_col='cat',
    time_stamp_col='time_stamp',
    kind='drift', 
    chunk_period='M')

TypeError: combine_drift_each_col_plot() got an unexpected keyword argument 'chunk_period'